In [1]:
!python3 -m pip install --upgrade pip
!git clone -b mixtral https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .
!pip install --upgrade transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.1 MB/s eta 0:00:00:00:010:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Cloning into 'mergekit'...
remote: Enumerating objects: 2002, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 2002 (delta 72), reused 66 (delta 42), pack-reused 1886
Receiving objects: 100% (2002/2002), 611.94 KiB | 4.93 MiB/s, done.
Resolving deltas: 100% (1369/1369), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.9.0 requires keras-core, which is not installed.
keras-nlp 0.12.1 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
conda 24.5.

In [2]:
from huggingface_hub import notebook_login as hfl
hfl()

In [11]:
MODEL_NAME = "Code-Mistral-Llama-7B-TIES"
OUTPUT_DIR = "merge"
CONFIG_NAME = "config"

In [12]:
config = """
models:
  - model: mistralai/Mistral-7B-v0.1  # no parameters necessary for base model
  - model: codellama/CodeLlama-7b-Instruct-hf
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
  int8_mask: true
dtype: float16
"""

In [13]:
with open(f"{CONFIG_NAME}.yaml", "w") as f:
    f.write(config)

In [14]:
mergeCommand = f"mergekit-yaml {CONFIG_NAME}.yaml {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"  

In [ ]:
!{mergeCommand}

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
  2%|▊                                        | 28/1457 [00:05<02:45,  8.63it/s]